# Energy Storage Systems (ESS)
This notebook is inspired by Chapter 7 of the following book:<br>
Soroudi, Alireza. Power system optimization modeling in GAMS. Vol. 78. Berlin/Heidelberg, Germany: Springer, 2017.

In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Energy storage systems (ESS) can be added to power systems in order to mitigate the uncertainties associated with renewable sources. The addition of ESS intorduces more complexity to traditonal power system optimization problems such as dynamic economic dispatch, optimal power flow, etc.

# 1. Addidtion of ESS to DED
**Problem Description:**<br>
The mathematical constraints to represent a DED problem with ESS would lokk like the following:
$$
\underset{P_{g,t},SOC_{t},P_{d}^{t},P_{c}^{t}}{min} \; TC = \sum_{g,t}^{} a_{g}P_{g,t}^{2} + b_{g}P_{g,t} + c_{g}
$$
$$
P_{g}^{min} \le P_{g,t} \le P_{g}^{max}
$$
$$
P_{g,t} - P_{g,t-1} \le RU_{g}
$$
$$
P_{g,t-1} - P_{g,t} \le RD_{g}
$$
$$
SOC_{t} = SOC_{t-1} + (P_{t}^{c}\eta_{c} - P_{t}^{d}/\eta_{d}) \Delta_{t}
$$
$$
P_{min}^{c} \le P_{t}^{c} \le P_{max}^{c}
$$
$$
P_{min}^{d} \le P_{t}^{d} \le P_{max}^{d}
$$
$$
SOC_{min} \le SOC_{t} \le SOC_{max}
$$
$$
\sum_{g}^{} P_{g,t} + P_{t}^{d} \ge L_{t} - P_{t}^{c}
$$

The following tables provide the operational specifications of the generators, and the battery.

In [2]:
gen_df = pd.DataFrame()
gen_df["g"] = ["g1", "g2", "g3", "g4"]
gen_df["$a_{g}$"] = [0.12, 0.17, 0.15, 0.19]
gen_df["$b_{g}$"] = [14.8, 16.57, 15.55, 16.21]
gen_df["$c_{g}$"] = [89, 83, 100, 70]
gen_df["$d_{g}$"] = [1.2, 2.3, 1.1, 1.1]
gen_df["$e_{g}$"] = [-5, -4.24, -2.15, -3.99]
gen_df["$f_{g}$"] = [3, 6.09, 5.69, 6.2]
gen_df["$P_{min}$"] = [28, 20, 30, 20]
gen_df["$P_{max}$"] = [200, 290, 190, 260]
gen_df["$RU_{g}$"] = [40, 30, 30, 50]
gen_df["$RD_{g}$"] = [40, 30, 30, 50]
gen_df

,g,$a_{g}$,$b_{g}$,$c_{g}$,$d_{g}$,$e_{g}$,$f_{g}$,$P_{min}$,$P_{max}$,$RU_{g}$,$RD_{g}$
0,g1,0.12,14.80,89,1.2,-5.00,3.00,28,200,40,40
1,g2,0.17,16.57,83,2.3,-4.24,6.09,20,290,30,30
2,g3,0.15,15.55,100,1.1,-2.15,5.69,30,190,30,30
3,g4,0.19,16.21,70,1.1,-3.99,6.20,20,260,50,50


In [6]:
batt_df = pd.DataFrame()
batt_df["$SOC_{0}(MW)$"] = [100]
batt_df["$SOC_{max}(MW)$"] = [300]
batt_df["$P_{max}^{d}(MW)$"] = [60]
batt_df["$P_{min}^{d}(MW)$"] = [0]
batt_df["$P_{max}^{c}(MW)$"] = [60]
batt_df["$P_{min}^{c}(MW)$"] = [0]
batt_df["$\eta_{c}$"] = [0.95]
batt_df["$\eta_{d}$"] = [0.90]
batt_df

,$SOC_{0}(MW)$,$SOC_{max}(MW)$,$P_{max}^{d}(MW)$,$P_{min}^{d}(MW)$,$P_{max}^{c}(MW)$,$P_{min}^{c}(MW)$,$\eta_{c}$,$\eta_{d}$
0,100,300,60,0,60,0,0.95,0.9
